# Introduction/Business Problem

I'm planning to move to Toronot from Montreal. I would like to move to the neighbourhood similar to where I currenlty live. For that purpose I would need to use Foursquare API to get the venues in my current neighbourhood. I would need to use clustering to determine cluster labels for Montreal.

Next, using Foursquare API, I'll fetch venues data for Toronoto. Instead of clustering I'll use classification methodology to classify Toronto's neighbourhoods given labels from Montreal neighbourhood clusterization. Then I'll choose same clusters in Toronto that I like in Montreal. And that would be my neighborhood of first choice to consider.


# Data 

1. I'll be using Foursquare API Venue

https://api.foursquare.com/v2/venues/explore

GET https://api.foursquare.com/v2/venues/explore

Returns a list of recommended venues near the current location. For more robust information about the venues themselves (photos/tips/etc.), please see our venue details endpoint.

### Request API 

| Name	| Example	| Description | 
|------|------|------|
|   ll  | 40.74224,-73.99386| required unless near is provided. Latitude and longitude of the user's location.|
|   near  | Chicago, IL| required unless ll is provided. A string naming a place in the world. If the near string is not geocodable, returns a failed_geocode error. Otherwise, searches within the bounds of the geocode and adds a geocode object to the response.|
|   radius  | 250| Radius to search within, in meters. If radius is not specified, a suggested radius will be used based on the density of venues in the area. The maximum supported radius is currently 100,000 meters.|

### Response Fields

| Field		| Description | 
|------|------|
| warning| Presents an object with a text field that contains a warning message, if applicable (i.e. not enough results, try doing X).|
| groups| A text name for the location the user searched, e.g. "SoHo".|
| headerLocation| A text name for the location the user searched, e.g. "SoHo".|
| id| A unique string identifier for this venue.|
| name|The best known name for this venue.|
| location|An object containing none, some, or all of address (street address), crossStreet, city, state, postalCode, country, lat, lng, and distance. All fields are strings, except for lat, lng, and distance. Distance is measured in meters. Some venues have their locations intentionally hidden for privacy reasons (such as private residences). If this is the case, the parameter isFuzzed will be set to true, and the lat/lng parameters will have reduced precision.|
| categories | An array, possibly empty, of categories that have been applied to this venue. One of the categories will have a primary field indicating that it is the primary category for the venue. For the complete category tree, see categories.|


    
2. I'll do same clustering exercise as I did for clustering Toronto neighbourhood, but this time for Montreal city.

Postal Codes for Montreal:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H
    
2. I'll use K-Nearest Neighbour Classifier to predict neghbour cluster given venues in 500m proximity.   